Nov 2018
Payne

 - Want to create a notebook to demo to J. Eastman some of the basic functionalities that (a) he'll need, and (b) that are available
    

### Standard python / jupyter imports 

In [2]:
import numpy as np
import os, sys
#import astropy as AP

### Imports of *HIGHLY DEVELOPMENTAL* (i.e. potentially wrong/slow) packages
 - This needs to be updated / generalized to get the damn things to install from pip ...

In [3]:
# >>> pip install mpcutilities 
import mpcutilities as MPCU
#import mpcadvancer  as MPCA

### Orbit specification

In [1]:
# Specify a Keplerian orbit
a,e,i,O,o,M       = (2.781870259390299, 0.07692192514945771, 0.18480538653567896, 1.4012469961929193, 1.237855673926063, -1.0950384781408455)
    
# Use kep2cartState to calculate the cartesian state from the specified elements 
CartState             = kc.kep2cartState(PHYS.GMsun , a,e,i,O,o,M)
print("CartState=", CartState.x, CartState.y, CartState.z, CartState.xd, CartState.yd, CartState.zd)

# Transform it back to Keplerian 
els    = kc.cart2kep(PHYS.GMsun , CartState)

# Check that the numbers look ok 
assert( np.allclose( els , np.array([a,e,i,O,o,M]) )) 


### Simulating Observations of an orbiting object

In [ ]:
# Specify the date of the proposed observation of the object
# - Midnight, Jan 1st, 2023, arbitrarily simulating data around this time ...
# - Orbit times use "TDB": https://en.wikipedia.org/wiki/Barycentric_Dynamical_Time
JD_TDB = 2459945.5 

# It will be useful to know the approximate mean anomaly of the earth at the above time 
# - This is to prevent us from simulating observations in stupid/unphysical directions
Observatory.getObservatoryPosition('500', JD_TDB) ##<<-- '500' is the geocenter
earthMeanAnom = True

# Specify a Cartesian state at a particular epoch
a,e,i,O,o,M       = (2.781870259390299, 0.07692192514945771, 0.18480538653567896, 1.4012469961929193, 1.237855673926063, earthMeanAnom)
CartState             = kc.kep2cartState(PHYS.GMsun , a,e,i,O,o,M)

# Specify where you are observing from (using observatory codes as short-hand)
JD_UTC  = 2459945.5 ## <<-- Observation taken at a time specified in UTC system: https://en.wikipedia.org/wiki/Coordinated_Universal_Time
obsCode = '806'     ## <<-- Observatory code: https://en.wikipedia.org/wiki/List_of_observatory_codes

# Calculate the topocentric RA & Dec of the object at the time of observation

### Advance a Cartesian state to a series of time-steps

In [2]:
# Specify a Cartesian state at a particular epoch
JD_TDB = 2459945.5, # <<-- Midnight, Jan 1st, 2023, arbitrarily simulating data around this time ...
a,e,i,O,o,M       = (2.781870259390299, 0.07692192514945771, 0.18480538653567896, 1.4012469961929193, 1.237855673926063, -1.0950384781408455)
CartState         = kc.kep2cartState(PHYS.GMsun , a,e,i,O,o,M)

# Specify some times of interest
targetTDBs = np.array(JD_TDB , JD_TDB + 10.0, 1.0)

# Advance the initial cartesian state to a set of cartesian states, one at each of the times of interest
ics = ORBIT_ICs(something) ##<<-- Inputs need to be specified 
a   = ADVANCE(ics)
t   = a.twobody(targetTDBs)


### Simulating observations of an orbiting object at a series of time-steps

In [3]:
# Specify a Cartesian state at a particular epoch

# Specify some times of interest

# Advance the initial cartesian state to a set of cartesian states, one at each of the times of interest

# Simulate the observations

### Define some convenience functions for generating "realistic" data sets

In [209]:

def subsequent_new_moon_dates(JD, N):
    '''
    # Convenience function to return the dates of the N new-moons after a specified JD
    # - Used in function(s) below to generate synthetic data
    # - Totally unimportant, why did I spend any time doing this ?!?
    '''
    # When is it new moon ?
    # http://aa.usno.navy.mil/cgi-bin/aa_phases.pl?year=2018&month=11&day=12&nump=50&format=p
    JD_of_a_new_moon = 2458459.805556 # 2018 Dec 07 07:20 
    # Approx ave synodic month 
    period           = 29.530588      # Good enough for government work: https://en.wikipedia.org/wiki/New_moon
    # Find next new moon after input date
    Nfullphases, remainderDays = (JD - JD_of_a_new_moon ) // period , (JD - JD_of_a_new_moon ) % period
    next_new_moon = JD + (period - remainderDays)
    # Generate array of next N-new-moons
    return np.arange( next_new_moon, next_new_moon + N*period, period )

def generate_darkTime_arrays(JD_UTC, Nlunations=3):
    '''
    # Convenience function to identify the periods of "dark-time" (away from full moon) after a given date
    # - Used in function(s) below to generate synthetic data
    # - Totally unimportant, why did I spend any time doing this ?!?
    '''
    # Find the new moons dates after JD_UTC
    newMoonNights = subsequent_new_moon_dates(JD_UTC, Nlunations)

    # Get the unbroken ~22+ night span(s) of dark(ish)-time centered-on the above new moon dates
    Ndark = 2*11
    darkNightArrays = []
    for i,n in enumerate(newMoonNights[:-1]):
        tStart = int(newMoonNights[i] - Ndark/2) 
        tEnd   = int(newMoonNights[i] + Ndark/2)
        if tStart > JD_UTC:
            darkNightArrays.append(  np.arange( tStart, tEnd ,1 ) )
        assert darkNightArrays != [], "nothing added to darkNightArrays ... "
    return darkNightArrays

def choose_nights_from_span(nightArray, Nnights, ArcLength):
    '''
    # Convenience function to choose randomly (with constraints) N-nights from a contoguous span 
    # - Used in function(s) below to generate synthetic data
    '''
    assert Nnights <= ArcLength, "Nnights is more than the specified ArcLength"
    assert Nnights <= len(nightArray), "Nnights is more than the available nights"
    ArcLength = int(ArcLength) 
    
    # First night must be within the first few (to allow arc-length constraint to be satisfied)
    firstNight       = np.random.choice(nightArray[: len(nightArray)- ArcLength ])
    selectedNights   = [firstNight]
    # If Nnights > 1, generate a total span that is approx equal to ArcLength 
    if Nnights > 1:
        lastNight        = firstNight + ArcLength  
        selectedNights.append(lastNight)
    # If Nnights > 2, add in extra nights 
    if Nnights > 2:
        additionalNights = np.random.choice( np.arange(firstNight + 1 , lastNight) , Nnights-2, replace=False)
        selectedNights.extend( list(additionalNights) )
    return np.sort(np.array(selectedNights))
                                          
def generate_synthetic_LSST_observations_SingleLunation(
        OrbitalCartesianState,
        t0_orb_JD_TDB = 2459945.5, # <<-- Midnight, Jan 1st, 2023, arbitrarily simulating data around this time ...
        t0_obs_JD_UTC = 2459945.5, # <<-- Midnight, Jan 1st, 2023, ... and assuming orbit specified around same time
        Ndets=6,                   # <<-- Total number of individual detections  
        Nnights=3,                 # <<-- Total number of nights with data on them
        ArcLength=10.,             # <<-- Number of nights separating start of data from end of data
        obsCode='806',             # <<-- Observatory code to use for LSST  
        obsError=0.1,              # <<-- Assumed astrometric uncertainty for LSST observations 
        outlier=True):             # <<-- Whether to add outlier observational errors for shits and giggles
    '''
    # Convenience function to generate synthetic observational data sets 
    # LSST discovery data will be *short*: 
    # - typically ~6 detections, 
    # - spread across ~3 individual nights, 
    # - with the span of the data ("arc length") being ~10 nights from start to end 
    '''
    # Assertions to avoid fuck-ups due to the assumptions I make below
    assert ArcLength < 20
    assert Nnights <= ArcLength
    assert Ndets >= Nnights*2 ## <<-- LSST requires >= 2 per night
    assert Ndets <= Nnights*4 ## <<-- No good reason, it just seems unnecessary for LSST ....
    
    # Get nights away from the full moon 
    # - N.B. "[0]" ... just selecting the first set of dark time ...
    darkNightArrays = generate_darkTime_arrays(t0_obs_JD_UTC)[0] 
    
    # Choose nights, ensuring a data-span approx equal to ArcLength 
    selectedNights = choose_nights_from_span(darkNightArrays, Nnights, ArcLength)
    
    # Specify number of observations on each night    
    # - To simplify, am having everything have the average, then add in extras here-and-there
    aveObsPerNight, additionalObsBeyondAve = Ndets // Nnights , Ndets % Nnights
    nightsToGetAdditionalObs = np.random.choice( selectedNights , additionalObsBeyondAve, replace=False)
                                          
    # Choose observation times for each night
    timeRange = (0.33,0.75) ##<<-- min & max JD_UTC, corresponds to approx 8pm & 6am
    selectedObservationJDUTCs = []
    for night in selectedNights:
        startTime = np.random.uniform( timeRange[0],timeRange[1]-1.01/24.) ## <<-- leave at least an hour at the end
        endTime   = np.random.uniform( startTime+1/24. ,  timeRange[1])    ## <<-- want the detections >= 1-hr apart 
        if night not in nightsToGetAdditionalObs:
            selectedObservationJDUTCs.extend( [night+startTime , night+endTime])
        else:
            extraTime = np.random.uniform( startTime ,  endTime )
            selectedObservationJDUTCs.extend( [night+startTime , night+extraTime , night+endTime])
    print("selectedObservationJDUTCs", selectedObservationJDUTCs)
    
    '''
    
    # Evolve the input OrbitalCartesianState to the required times and simulate the observations 
    
    
    # Decide whether to add extra errors to a few data points because no one has a clue what error bars really mean
    if outlier:
        Noutliers = np.random.int(1,int(len(OBS)/2)) ## <<-- At least one, but less than half
        Outliers  = np.random.choice( np.arange(len(OBS)) , Noutliers, replace=False)
    # Add  errors
    for i,O in enumerate(OBS):
        if i in Outliers:
            factor = 2.0
        else:
            factor = 1.0
        O.RA  += np.random.normal(0.0, factor*obsError)
        O.DEC += np.random.normal(0.0, factor*obsError)
        
    # Return cartesian state(s) as well as observations ...
    return OBS
    '''
    
    
    

### Plot results from above convenience function

In [208]:
# Define input cartesian state 
OrbitalCartesianState = True

# Generate synthetic observations 
_ = generate_synthetic_LSST_observations_SingleLunation(CartState)

# Make a plot of the cartesian position(s) 

# Make a plot of the RA & Dec (with & without uncertainties?)

selectedObservationJDUTCs [2459959.5102544706, 2459959.6515298695, 2459964.394071052, 2459964.530711493, 2459969.5560183674, 2459969.6396634015]
